# Noticias

In [11]:
import investpy as ivp
import numpy as np
import datetime
import time
import pytz

In [2]:
today=datetime.datetime.today().date()
timezone=int(int(time.strftime("%z", time.gmtime()))/100)

In [3]:
hora=datetime.datetime.now().time().strftime("%H:%M")
hora

'20:38'

In [4]:
calendario=ivp.economic_calendar(time_zone=f"GMT +{timezone+2}:00",importances=["high","medium"],from_date=f"02/06/2024",to_date=f"03/06/2024")
calendario=calendario.drop(["id","actual","forecast","previous"],axis=1)
calendario=calendario.iloc[np.where(calendario["date"]==f"03/06/2024")]
calendario=calendario.drop(calendario[calendario['time'] == 'All Day'].index)
calendario

,date,time,zone,currency,importance,event
7,03/06/2024,02:50,japan,JPY,medium,Capital Spending (YoY) (Q1)
8,03/06/2024,04:45,china,CNY,medium,Caixin Manufacturing PMI (May)
9,03/06/2024,10:00,türkiye,TRY,medium,CPI (MoM) (May)
10,03/06/2024,10:00,türkiye,TRY,medium,CPI (YoY) (May)
11,03/06/2024,10:15,spain,EUR,medium,HCOB Spain Manufacturing PMI (May)
12,03/06/2024,10:30,switzerland,CHF,medium,procure.ch PMI (May)
13,03/06/2024,10:45,italy,EUR,medium,HCOB Italy Manufacturing PMI (May)
14,03/06/2024,10:50,france,EUR,medium,HCOB France Manufacturing PMI (May)
15,03/06/2024,10:55,germany,EUR,medium,HCOB Germany Manufacturing PMI (May)
16,03/06/2024,11:00,poland,PLN,medium,GDP (QoQ) (Q1)


In [15]:
ts=datetime.datetime(2024,1,1)
#print(pytz.all_timezones) 
spain=pytz.timezone("Europe/Madrid")


In [5]:
calendario=ivp.economic_calendar(time_zone=f"GMT +{timezone+2}:00",importances=["high","medium"],from_date=f"{(today-datetime.timedelta(days=1)).strftime("%d/%m/%Y")}",to_date=f"{today.strftime("%d/%m/%Y")}")
calendario=calendario.drop(["id","actual","forecast","previous"],axis=1)
calendario=calendario.iloc[np.where(calendario["date"]==f"{today.strftime("%d/%m/%Y")}")]
calendario=calendario.iloc[np.where(calendario["importance"]=="high")]
calendario=calendario.drop(calendario[calendario['time'] == 'All Day'].index)
calendario["time"]=[datetime.datetime.strptime(item,"%H:%M") for item in calendario["time"]]
calendario

,date,time,zone,currency,importance,event
20,03/06/2024,1900-01-01 16:45:00,united states,USD,high,S&P Global US Manufacturing PMI (May)
23,03/06/2024,1900-01-01 17:00:00,united states,USD,high,ISM Manufacturing PMI (May)
24,03/06/2024,1900-01-01 17:00:00,united states,USD,high,ISM Manufacturing Prices (May)


medium = no operar 2 min antes y 5 min despues

high = no operar 10 min antes y 20 min despues

si esta en este intervalo liquidar ordenes y posiciones

MetaTrader5 invierno GMT +2 | Verano GMT +3

España  invierno GMT +1 | Verano GMT +2

In [103]:
hora=datetime.datetime.today()

if len(calendario)>0:
    start=time.time()
    #coincidencias=list(i for i in calendario["time"] if (i-datetime.timedelta(hours=2)).time()<=hora<=(i+datetime.timedelta(hours=2)).time())
    coincidencias=calendario.loc[((hora-datetime.timedelta(hours=2)).time()<=calendario['time']) & ((calendario['time']<=hora+datetime.timedelta(hours=2)).time())]
    print(coincidencias)
    for i in range(10):
        print((time.time()-start)*(10**6)) #otro=996

    print()
    if len(coincidencias)>0:
        for i in coincidencias:
            print(i)
            print()
            #calendario=calendario.drop(i,axis=1)
            #calendario = calendario[calendario.time != i]
        print("No operar")


AttributeError: 'Series' object has no attribute 'time'

comprobar si alguno de la lista, el tiempo es cercano al tiempo actual

si el evento es high importance, cerrar posiciones y esperar

# -----------------------------------------------------------

# Ordenes

In [1]:
import MetaTrader5 as mt
import pandas as pd
from utils import *
import time

In [2]:
mt.initialize()

True

In [3]:
login=80284478
password="*iWcMw3k"
server="MetaQuotes-Demo"

mt.login(login, password, server)

True

In [4]:
if mt.market_book_add('EURUSD'):
    items = mt.market_book_get('EURUSD')
    print(len(items))
    #for i in sorted(items,key=lambda x:x[2],reverse=True): #ordenar por volumen
    for i in items:
        print(i)

    mt.market_book_release('EURUSD')
else:
    print("mt5.market_book_add('EURUSD') failed, error code =",mt.last_error())

0


In [8]:
start=time.time()
fecha_inicio=datetime(2024,3,8)
fecha_final=datetime(2024,3,9)
tick_data = pd.DataFrame(mt.copy_ticks_range("SP500m", fecha_inicio,fecha_final, mt.COPY_TICKS_ALL))
print("Primer tick: ",datetime.fromtimestamp(tick_data.iloc[0]["time"])," precio: ",tick_data.iloc[0]["bid"],tick_data.iloc[0]["ask"])
print("Ultio tick: ",datetime.fromtimestamp(tick_data.iloc[-1]["time"])," precio: ",tick_data.iloc[-1]["bid"],tick_data.iloc[-1]["ask"])
print(round(time.time()-start,4),"s")

tick_data["time"]=[datetime.fromtimestamp(item) for item in tick_data["time"]]

tick_data

IndexError: single positional indexer is out-of-bounds

In [53]:
#tick_data[31000:31200]
for i in tick_data[31200:32000].iterrows():
    print(i[1][0:3])
    print()

time    2024-04-02 17:00:54
bid                 1.07525
ask                 1.07528
Name: 31200, dtype: object

time    2024-04-02 17:00:54
bid                 1.07526
ask                 1.07529
Name: 31201, dtype: object

time    2024-04-02 17:00:54
bid                 1.07526
ask                 1.07528
Name: 31202, dtype: object

time    2024-04-02 17:00:55
bid                 1.07526
ask                 1.07529
Name: 31203, dtype: object

time    2024-04-02 17:00:55
bid                 1.07525
ask                 1.07527
Name: 31204, dtype: object

time    2024-04-02 17:00:56
bid                 1.07524
ask                 1.07527
Name: 31205, dtype: object

time    2024-04-02 17:00:57
bid                 1.07522
ask                 1.07525
Name: 31206, dtype: object

time    2024-04-02 17:00:57
bid                 1.07522
ask                 1.07524
Name: 31207, dtype: object

time    2024-04-02 17:00:58
bid                 1.07523
ask                 1.07526
Name: 31208, dtype: 

In [4]:
rates = mt.copy_rates_range("BTCUSD", mt.TIMEFRAME_M5, datetime(2024, 3, 8),datetime(2024, 3,8))
rates

In [7]:
hoy=datetime.today().date()
hoy

datetime.date(2024, 8, 1)

In [16]:
datetime.fromordinal(datetime.today().toordinal())

datetime.datetime(2024, 8, 1, 0, 0)

In [14]:
tick_data=pd.DataFrame(mt.copy_ticks_range("EURUSD", datetime.today(),datetime.today()+timedelta(1), mt.COPY_TICKS_ALL))
tick_data["time"]=[datetime.fromtimestamp(item) for item in tick_data["time"]]
print(tick_data)

                    time      bid      ask  last  volume       time_msc  \
0    2024-08-01 13:20:08  1.07835  1.07839   0.0       0  1722511208455   
1    2024-08-01 13:20:08  1.07837  1.07839   0.0       0  1722511208958   
2    2024-08-01 13:20:11  1.07838  1.07840   0.0       0  1722511211055   
3    2024-08-01 13:20:13  1.07839  1.07843   0.0       0  1722511213165   
4    2024-08-01 13:20:13  1.07848  1.07850   0.0       0  1722511213668   
...                  ...      ...      ...   ...     ...            ...   
7867 2024-08-01 16:20:03  1.07928  1.07930   0.0       0  1722522003641   
7868 2024-08-01 16:20:06  1.07924  1.07929   0.0       0  1722522006156   
7869 2024-08-01 16:20:06  1.07923  1.07925   0.0       0  1722522006650   
7870 2024-08-01 16:20:07  1.07924  1.07926   0.0       0  1722522007245   
7871 2024-08-01 16:20:08  1.07923  1.07925   0.0       0  1722522008348   

      flags  volume_real  
0      1030          0.0  
1      1026          0.0  
2      1030       

In [40]:
datetime.datetime.now().hour

19

In [39]:
import datetime
utc_datetime = datetime.datetime.now(datetime.timezone.utc)

utc_to_local_datetime = utc_datetime.astimezone()
utc_2_local_iso_str = datetime.datetime.strftime(utc_to_local_datetime, "%Y-%m-%dT%H:%M:%S.%f")[:-3]

print( f"utc to local dt: {utc_2_local_iso_str}, tzname: {utc_to_local_datetime.tzname()}" )
print( f"utc to local dt: {utc_to_local_datetime.isoformat()}" )

2024-08-01 19:02:20.252273+02:00
utc to local dt: 2024-08-01T19:02:20.252, tzname: Hora de verano romance
utc to local dt: 2024-08-01T19:02:20.252273+02:00


In [5]:
data_5m=pd.DataFrame(mt.copy_rates_range("EURUSD", mt.TIMEFRAME_M5, datetime(2024,8,1),datetime(2024,8,2)))
data_5m["time"]=[datetime.fromtimestamp(item) for item in data_5m["time"]]
print(data_5m)

                   time     open     high      low    close  tick_volume  \
0   2024-08-01 00:00:00  1.08261  1.08311  1.08260  1.08298          275   
1   2024-08-01 00:05:00  1.08298  1.08313  1.08282  1.08282          253   
2   2024-08-01 00:10:00  1.08282  1.08328  1.08276  1.08305          270   
3   2024-08-01 00:15:00  1.08305  1.08324  1.08298  1.08303          278   
4   2024-08-01 00:20:00  1.08304  1.08307  1.08273  1.08273          285   
..                  ...      ...      ...      ...      ...          ...   
189 2024-08-01 15:45:00  1.07877  1.07878  1.07835  1.07837          131   
190 2024-08-01 15:50:00  1.07837  1.07837  1.07810  1.07815          175   
191 2024-08-01 15:55:00  1.07812  1.07849  1.07806  1.07843          148   
192 2024-08-01 16:00:00  1.07843  1.07888  1.07807  1.07878          302   
193 2024-08-01 16:05:00  1.07878  1.07919  1.07862  1.07916          223   

     spread  real_volume  
0         0            0  
1         1            0  
2     

In [9]:
start=time.time()
tick_data = pd.DataFrame(mt.copy_ticks_range("ND100m", datetime(2024, 3, 8),datetime(2024, 3,8), mt.COPY_TICKS_ALL))
print("Primer tick: ",datetime.fromtimestamp(tick_data.iloc[0]["time"])," precio: ",tick_data.iloc[0]["bid"],tick_data.iloc[0]["ask"])
print("Ultio tick: ",datetime.fromtimestamp(tick_data.iloc[-1]["time"])," precio: ",tick_data.iloc[-1]["bid"],tick_data.iloc[-1]["ask"])
print(round(time.time()-start,4),"s")
tick_data

IndexError: single positional indexer is out-of-bounds

In [11]:
contador=0
deviation=2.0
resultado,contador=order("EURUSD",mt.symbol_info_tick("EURUSD").ask,1.06,1.1,contador,mt.TRADE_ACTION_DEAL,mt.ORDER_TYPE_BUY,deviation,1.0,position=None)
print(resultado)

OrderSendResult(retcode=10009, deal=2346931059, order=2372822759, volume=1.0, price=1.0814, bid=1.08138, ask=1.0814, comment='Request executed', request_id=4148549708, retcode_external=0, request=TradeRequest(action=1, magic=0, order=0, symbol='EURUSD', volume=1.0, price=1.0814, stoplimit=0.0, sl=1.06, tp=1.1, deviation=20, type=0, type_filling=1, type_time=0, expiration=0, comment='Order number: 0', position=0, position_by=0))


In [6]:
def liquidate(deviation):
    for i in mt.positions_get():
        if i.type==1: # venta
            resultado,contador=order(i.symbol,mt.symbol_info_tick(i.symbol).ask,0.0,0.0,0,mt.TRADE_ACTION_DEAL,mt.ORDER_TYPE_BUY,deviation,i.volume,position=i.ticket)
        elif i.type==0: #compra
            resultado,contador=order(i.symbol,mt.symbol_info_tick(i.symbol).bid,0.0,0.0,0,mt.TRADE_ACTION_DEAL,mt.ORDER_TYPE_SELL,deviation,i.volume,position=i.ticket)

In [7]:
liquidate()
mt.positions_get()

()